This is a draft version to get the MinION reads zipped up a certain way and moved over to a specific HPC for basecalling.

Idea is to point the program towards the reads directory of MinKNOW, zip them up and move them over to a server

In [4]:
#for now name the initial BASEFOLDER, BACKUPFOLDER, NUMBEROFFOLDERS, SEQUENCINGRUN, RTARGETFOLDER
#
#later this will come from the config %%file
CONFIGFILE = '/mnt/d/Data/MinION_scripts/ZipTra.cfg'

In [5]:
import re
import os
import subprocess
import configparser 
import argparse
import tarfile

In [ ]:
parser = argparse.ArgumentParser(description="This is a script to get the MinION reads zipped up a certain way and moved over to a specific HPC for basecalling. Idea is to point the program towards the reads directory of MinKNOW, zip them up and move them over to a server")

In [ ]:
parser.add_argument("configfile", help="provide the directory of a config file which contain all the informations. A sample was provided in the same pakage of this script.")

CONFIGFILE = args.configfile

In [6]:
Config = configparser.ConfigParser()
Config.read(CONFIGFILE)

['/mnt/d/Data/MinION_scripts/ZipTra.cfg']

In [7]:
def ConfigSectionMap(section):
    dict1 = {}
    options = Config.options(section)
    for option in options:
        try:
            dict1[option] = Config.get(section, option)
            if dict1[option] == -1:
                DebugPrint("skip: %s" % option)
        except:
            print("exception on %s!" % option)
            dict1[option] = None
    return dict1

In [8]:
DIRS = ConfigSectionMap('Directories')
BACKUPFOLDER = DIRS['backupfolder']
BASEFOLDER = DIRS['basefolder']
RTARGETFOLDER = DIRS['rtargetfolder']

In [9]:
SETUP = ConfigSectionMap('Zip settings')
NUMBEROFFOLDERS = int(SETUP['numberoffolders'])
SEQUENCINGRUN = SETUP['sequencingrun']

In [10]:
#now check if basefolder is there
if not os.path.exists(BASEFOLDER):
    print('Check basefolder')
    exit
if not os.path.exists(BACKUPFOLDER):
    print('Backup folder %s does not exist. Creating it.' %BACKUPFOLDER)
    os.mkdir(BACKUPFOLDER)

In [11]:
#get all the the folders witht the SEQUENCINGRUN ID in the basefolder
_id = re.compile(SEQUENCINGRUN, re.IGNORECASE)

In [12]:
#now get all the folders belonging to this sequencing %run
read_folder_list = [os.path.join(BASEFOLDER, x) for x in os.listdir(BASEFOLDER)]
flowcellfolder = [x for x in read_folder_list if _id.search(x) != None]

In [13]:
#now check if in the target folder is fast5 folder
def check_fast5(x):
    if not os.path.exists(os.path.join(x, 'fast5')):
        return False
    elif not os.path.isdir(os.path.join(x, 'fast5')):
        return False
    else:
        fast5path = os.path.join(x, 'fast5')
        #now check if the first folder contains fast5 files
        subfolders = [os.path.join(fast5path, x) for x in os.listdir(fast5path)]
        if len(subfolders) == 0:
            return False
        else:
            content = [x for x in os.listdir(subfolders[0]) if x.endswith('.fast5')]
            if len(content) == 0:
                return False
            else:
                #print(content)
                return True

In [3]:
some = 'something'

In [13]:
some[1:50000]

'omething'

In [14]:
def tar_zip(fast5_folder,backupfolder, start, end):
    folder = fast5_folder.split('/')[-2]
    tarfile_name = os.path.join(backupfolder,'%s_%s_%s.tar.gz' % \
                                (folder, start, end))
    tar = tarfile.open(tarfile_name, "w:gz")
    source_folders = [os.path.join(fast5_folder, x)\
                      for x in os.listdir(fast5_folder)]
    source_folders = source_folders[start:end+1]
    for source_folder in source_folders:
        tar.add(source_folder)
    tar.close()             
    

In [15]:
def zip_and_backup(x, number, backupfolder):
    fast5path = os.path.join(x, 'fast5')
    #get the runs folder id
    folder = x.split('/')[-1]
    subfolders = [os.path.join(fast5path, x) for x in os.listdir(fast5path)]
    noffolders = len(subfolders)
    #noffolders = 300
    print("We have %i of folders to zip up." % noffolders) 
    if noffolders <= number:
        start = 0
        end = noffolders - 1
        tar_zip(fast5path, backupfolder, start, end)
    #elif noffolders > number:
    #    if number == 1:
    #        starts = [x for x in range(0, noffolders, number)]
    #        ends = starts
    #        start_end_zip = zip(starts, ends)
    #        for x in start_end_zip:
    #            start, end = x
    #            tar_zip(fast5path,backupfolder, start, end)
    else:
        start = 0
        while start < noffolders -number:
            end = start + number -1
            tar_zip(fast5path, backupfolder, start, end)
            start = start + number
        start = end + 1
        end = noffolders -1
        tar_zip(fast5path, backupfolder, start, end)

        

In [16]:
#now zip up all the folders
for x in flowcellfolder:
    if check_fast5(x) == True:
        zip_and_backup(x, NUMBEROFFOLDERS,  BACKUPFOLDER)
    else:
        print('%s does not contain any fast5 files in folder 0')

We have 5 of folders to zip up.


In [14]:
os.getcwd()

'/mnt/d/Data/reads/workspace/20171221_1103_20171221_FAH34985/fast5'

In [ ]:
def scp_zip(file, destination):
    command_list = []
    command_list.append('scp ')
    command_list.append('%s '%file)
    command_list.append('%s'%destination)
    command = ' '.join(command_list)
    #here add subprocess
    print(command)
    command_stderr = subprocess.check_output(command, shell=True, stderr=subprocess.STDOUT)

In [ ]:
for file in [os.path.join(BACKUPFOLDER, x) for x in BACKUPFOLDER if x.endswith('tar.gz')]:
    print(file)
    scp_zip(file, RTARGETFOLDER)

In [ ]:
for file in [os.path.join(BACKUPFOLDER, x) for x in os.listdir(BACKUPFOLDER) if x.endswith('tar.gz')]:
    print(file)
    scp_zip(file, RTARGETFOLDER)